In [3]:
from pathlib import Path
import json
import pandas as pd
import os

In [4]:
DRIVE_PATH = Path("/mnt/h/")
KODF_PATH = DRIVE_PATH / "kodf_release"
DFDC_PATH = DRIVE_PATH / "dfdc_extracted"

os.listdir(KODF_PATH)

['KoDF_Terms_of_Use.pdf',
 'original_video_metadata.csv',
 'README.md',
 'synthesized_video_metadata.csv',
 'attacked_videos',
 'original_videos',
 'synthesized_videos']

# Pre-Proc KODF

In [10]:
kodf_synthesized_metadata = pd.read_csv(KODF_PATH / "synthesized_video_metadata.csv")
kodf_synthesized_videos_to_keep = kodf_synthesized_metadata[~kodf_synthesized_metadata.model.isin(["fo", "audio-driven"])]
kodf_synthesized_videos_to_keep['video_path'] = KODF_PATH / "synthesized_videos" / kodf_synthesized_videos_to_keep['model'] / kodf_synthesized_videos_to_keep['date_time'].str.replace("-", "") / kodf_synthesized_videos_to_keep['target_person_id'] / kodf_synthesized_videos_to_keep['video']
kodf_synthesized_videos_to_keep['video_path'] = kodf_synthesized_videos_to_keep['video_path'].apply(lambda x: str(x))
kodf_synthesized_videos_to_keep['label'] = "FAKE"
kodf_synthesized_videos_to_keep.drop(["sex", "date_time", "model", "target_person_id", "target_video", "source_person_id"], axis=1, inplace=True)
kodf_synthesized_videos_to_keep.set_index("video", inplace=True)

kodf_original_metadata = pd.read_csv(KODF_PATH / "original_video_metadata.csv")
kodf_original_metadata['video_path'] = KODF_PATH / "original_videos" / kodf_original_metadata["person_id"] / kodf_original_metadata["video"]
kodf_original_metadata['video_path'] = kodf_original_metadata['video_path'].apply(lambda x: str(x))
kodf_original_metadata['label'] = "REAL"
kodf_original_metadata.drop(["sex", "date_time", "studio", "person_id"], axis=1, inplace=True)
kodf_original_metadata.set_index("video", inplace=True)

kodf_combined = pd.concat([kodf_synthesized_videos_to_keep, kodf_original_metadata])
kodf_dict = kodf_combined.to_dict(orient="index")

with open(KODF_PATH / "master_metadata.json", 'w') as f:
    json.dump(kodf_dict, f)

/tmp/ipykernel_13675/391674642.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kodf_synthesized_videos_to_keep['video_path'] = KODF_PATH / "synthesized_videos" / kodf_synthesized_videos_to_keep['model'] / kodf_synthesized_videos_to_keep['date_time'].str.replace("-", "") / kodf_synthesized_videos_to_keep['target_person_id'] / kodf_synthesized_videos_to_keep['video']
/tmp/ipykernel_13675/391674642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kodf_synthesized_videos_to_keep['video_path'] = kodf_synt

In [ ]:
# Uncomment for processing 

# parts = [i for i in range(50)]
# master_metadata = {}
# for part in parts:
#     print(f"Processing Part: {part}")
#     folder_name = f"dfdc_train_part_{str(part).zfill(2)}"
#     inner_folder_name = f"dfdc_train_part_{part}"
#     current_part_path = DFDC_PATH / folder_name / inner_folder_name
#     metadata_path = current_part_path / "metadata.json" 
#     part_meta = {}
#     with open(metadata_path) as f:
#         meta_data = json.load(f)
#     for video, data in meta_data.items():
#         video_path = current_part_path / video
#         if not os.path.exists(video_path) == True:
#             print("CURRENT VIDEO PATH", video_path, video, data)
#             continue
#         del data['split']
#         if 'original' in data.keys():
#             del data['original']

#         data['video_path'] = str(video_path)
#         part_meta[video] = data
#     master_metadata.update(part_meta)

# with open(DFDC_PATH / "master_metadata.json", 'w') as f:
#     json.dump(master_metadata, f)

In [13]:
kodf_df = pd.read_json(KODF_PATH / "master_metadata.json", orient="index")
dfdc_df = pd.read_json(DFDC_PATH / "master_metadata.json", orient="index")

In [18]:
final_dataset = pd.concat([kodf_df, dfdc_df])
final_dataset.to_json("metadata.json", orient="index")